# Importing the necessary libraries

In [4]:
import pandas as pd
import numpy as np
import joblib
from keras.models import load_model
from datetime import datetime,date
from dateutil.relativedelta import relativedelta

# Reading and preprocessing of dataset

In [5]:
today_date = date.today().strftime("%Y-%m-%d")

In [6]:
dataset = pd.read_csv(f"D:\\Projects\\Jupyter_Lab\\Stock_market_predictor\\stock_prices_dataset\\Stocks_train_{today_date}.csv")

In [7]:
dataset_test = dataset.pivot(index='Date',columns='Stock',values='Close').reset_index()

In [8]:
# Feature Scaling
num_cols = ['ASHOKLEY','CANBK','LICI','ONGC','SBIN']
sc = joblib.load('D:\\Projects\\Jupyter_Lab\\Stock_market_predictor\\Stock_predictor_BSE_2025\\model_files\\min_max_scaler.joblib')
test_data = sc.transform(dataset_test[num_cols])

In [13]:
dataset_test[len(dataset_test)-60:len(dataset_test)]

(60, 6)

In [18]:
# Creating a data structure with 60 time steps and 1 output
X_test = test_data[len(test_data)-60:len(test_data),:]

In [22]:
X_test = X_test.reshape(1,60,-1)

# Load the model and predict

In [23]:
regressor_load = load_model('D:\\Projects\\Jupyter_Lab\\Stock_market_predictor\\Stock_predictor_BSE_2025\\model_files\\stock_predictor.keras')

In [32]:
today_preds = regressor_load.predict(X_test)
today_preds = sc.inverse_transform(today_preds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


In [33]:
today_preds_pd = pd.DataFrame(today_preds,columns=num_cols)
today_preds_pd['Date'] = today_date
today_preds_pd = today_preds_pd[['Date']+num_cols]

In [34]:
today_preds_pd

,Date,ASHOKLEY,CANBK,LICI,ONGC,SBIN
0,2025-06-02,229.852448,118.996811,933.962524,255.697906,790.096008


In [35]:
!git config http.postBuffer 524288000